In [1]:
%pip install fastapi pydantic toml

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import re
import tomllib
import functools
from typing import Literal

import ipywidgets as widgets
from IPython.display import display

from dotenv import load_dotenv

load_dotenv()

from rs import Message, Config, chat, skill_check
import d20

action_pattern = re.compile(r"(\d+). *(.+)")
skill_pattern = re.compile(r"[\w_]+")
skill_difficulty_pattern = re.compile(r"(easy|medium|hard)")

def next_round(b):
    global current_round, initial_messages
    current_round += 1
    initial_messages = config.initial_messages(current_round, config.metadata)

    print(initial_messages[-1].content)
    display(get_controls(initial_messages[-1]))

def get_user_message_tail():
    count = 0
    for msg in initial_messages:
        if msg.role == "user":
            count += 1

    is_final =  ", FINAL ROUND" if  current_round == len(config.rounds) else ""
    return f"TURN: {count + 1} {is_final}"
    

def do_chat(role:Literal["user", "system", "assistant"], content: str):
    print(f"{content}\n\n")

    initial_messages.append(Message(role=role, content=content))
    msg = chat(initial_messages, lambda delta: print(delta, end=""))

    initial_messages.append(msg)
    display(get_controls(msg))


def on_skill(b, skill:str, difficulty:str):
    content = skill_check(skill, difficulty, config.metadata)
    content += f"\n{get_user_message_tail()}"
    do_chat("user", content)

def on_action(b, index:int):
    content = f"I select option {index}. {get_user_message_tail()}"
    do_chat("user", content)

def on_custom_input(b, input:widgets.Text):
    content = f"{input.value}. {get_user_message_tail()}"
    do_chat("user", content)

def on_custom_action(b):
    hbox = widgets.HBox(layout=widgets.Layout(width="100%"))
    action_text = widgets.Text(layout=widgets.Layout(width="100%"))
    submit = widgets.Button(description="SUBMIT", layout=widgets.Layout(width="fit-content"))
    submit.on_click(functools.partial(on_custom_input, input=action_text))
    hbox.children += (action_text, submit) 
    display(hbox)

def get_controls(msg:Message) -> widgets.Box:
    dict = msg.dict()
    buttons = []
    if "possible actions" in dict.keys():
        matches = action_pattern.finditer(dict["possible actions"])
        for match in matches:
            btn = widgets.Button(description=match.group(1), tooltip=match.group(2), layout=widgets.Layout(width="auto"))
            btn.on_click(functools.partial(on_action, index=int(match.group(1))))
            buttons.append(btn)
    elif "skill" in dict.keys() and "difficulty" in dict.keys():
        match = skill_pattern.match(dict["skill"])
        skill = match.group(0)
        match = skill_difficulty_pattern.match(dict["difficulty"])
        difficulty = match.group(0)
        skill_button = widgets.Button(description=f"{skill.upper()} [{difficulty.upper()}]", layout=widgets.Layout(width="auto"))
        skill_button.on_click(functools.partial(on_skill, skill=skill, difficulty=difficulty))
        buttons.append(skill_button)
    elif "summary" in dict.keys():
        # add summary as a memory to the metadata
        # it will be used in the next round.
        config.metadata["memories"] = []
        config.metadata["memories"].append(dict["summary"])
        next_btn = widgets.Button(description="NEXT ROUND", layout=widgets.Layout(width="auto"))
        next_btn.on_click(next_round)
        buttons.append(next_btn)
    
    custom_action_button = widgets.Button(description="CUSTOM")
    custom_action_button.on_click(on_custom_action)
    buttons.append(custom_action_button)

    return widgets.HBox(buttons)

# open config file
config_name = "data/config_romance.toml"
config: Config | None = None
with open(config_name, "rb") as f:
    data = tomllib.load(f)
    config = Config(**data)

current_round = 0
initial_messages = config.initial_messages(current_round, config.metadata)

print(initial_messages[-1].content)
display(get_controls(initial_messages[-1]))

火车行驶在前往维也纳的旅程上，嘈杂的车厢和闷热的天气让你不免有些烦躁。当你抬头时，一个美丽的女孩映入你的眼帘——她坐在对面，专心致志地阅读手中的《致青年诗人的信》，阳光正从车窗斜斜地洒在一个美丽女孩的脸颊上，为她的轮廓镀上了一层金边。她身边散落着一些画具和一本摊开的素描本，显然是个酷爱艺术的人。虽然周围人声鼎沸，但她似乎完全沉浸在自己的世界中。就在她不经意地调整耳机时，短暂地与你目光相接，那一刻，时间仿佛静止。你感到一种无法抗拒的冲动，想要跨过车厢中的距离，用一句话打破这份静谧与隔阂。

possible actions:
1. "《致青年诗人的信》，你也喜欢里尔克吗？"
2. "嗨，可以借一下你的纸笔吗？"
3. "这车厢真是吵闹，你怎么能在这样的环境下这么专注？"


I select option 1. TURN: 1 


"《致青年诗人的信》，你也喜欢里尔克吗？" 你试图掩饰自己的紧张，微笑着开口。

女孩抬头，有些惊讶地看着你，然后轻轻点头，"嗯，是的。里尔克的诗，总能触动我内心最深处的某些东西。" 她的声音很小，带着一点不确定，仿佛是在对陌生人说话时的害羞。

你注意到她说话时的小动作，她轻轻咬了咬下唇，一瞬间的害羞让她看起来更加可爱。这让你更加想要继续这段对话，了解她更多。

possible actions:
1. "我完全同意，他的诗有种魔力，能深深打动人心。你最喜欢他的哪首诗？"
2. "我是杰西，美国人，正在环欧洲旅行。很高兴遇见像你这样有趣的人。"
3. "这本书里有没有什么特别推荐的诗？我也很喜欢读诗，但对里尔克的作品不是很熟悉。"

I select option 2. TURN: 2 


"我是杰西，美国人，正在环欧洲旅行。很高兴遇见像你这样有趣的人。" 你伸出手，带着一丝自信的笑容。

女孩略显害羞地回应你的微笑，她轻轻地和你握了握手，"我叫席琳，是个学习艺术史的学生。很高兴遇见你，杰西。" 她的声音逐渐变得更加自然，虽然还是有点小声，但你能感受到她试图克服内心的害羞。

她的笑容里带着一点点好奇，"环欧洲旅行听起来很有趣，你去过哪些地方呢？" 她的表情变得更加放松，好像你们的对话给了她一种安心的感觉。

possible actions:
1. "我已经去过几个城市了，像是巴黎、柏林、布达佩斯...每个地方都有它独特的美。"
2. "说实话，我最期待的就是维也纳，因为音乐和艺术的氛围。你呢，席琳，对艺术史有什么特别的兴趣吗？"
3. "其实我觉得旅行的意义不在于你去了哪里，而是在于旅途中遇到的人和事。比如说，现在和你的这次邂逅。"

I select option 3. TURN: 3 


"其实我觉得旅行的意义不在于你去了哪里，而是在于旅途中遇到的人和事。比如说，现在和你的这次邂逅。" 你真诚地说着，试图捕捉她的反应。

席琳的脸上露出了微笑，眼睛弯成了月牙形，似乎你的话语深深触动了她。"我非常同意你的看法，杰西。人与人之间的相遇，有时候比目的地本身更加珍贵。" 她的声音里充满了温暖和理解，她的态度也显得更加放松和自然。

她顿了一下，然后继续说道，"我之前去布达佩斯是看望我的祖母，现在在回巴黎大学的路上。虽然这次旅行主要是为了家庭，但我总是试图在每次旅行中寻找一些特别的体验和遇见一些有趣的人。" 席琳的笑容中透露出一种对生活的热爱和对未知的好奇。

随着对话的深入，你发现她开始更频繁地与你目光交汇，笑容中充满了温暖和友好。这次偶遇似乎给你们两个人都带来了意想不到的欢乐和思考。

game master:
-|context of the round|-
summary: 在这个回合里，杰西和席琳在火车上的偶遇逐渐从陌生人变成了有趣的旅途伙伴。从最初的文学兴趣的共鸣，到杰西的自我介绍，再到对旅行真正意义的探讨，两人之间的对话不仅打破了最初的尴尬，还激发了彼此对对方的好奇心和兴趣。席琳从一开始的害羞不安，逐步变得更加自在和开放，开始主动分享她的旅行经历和对生活的看法。两人在回合结束时已经建立起了一种微妙而美好的连接，为之后的互动埋下了温暖的种子。

随着火车轻轻摇晃，静静地朝着维也纳前行，你和席琳的对话逐渐深入。

席琳的眼睛闪烁着好奇的光芒，她微微倾身向你，带着兴趣盎然的神情问道：“你的环游欧洲之旅中有没有遇到什么特别有意思的事情？”她的声音中充满了期待，似乎在想象那些她未曾经历的冒险。

你笑了笑，手指轻敲着车窗边的桌面，思考从何讲起，然后兴奋地回应：“当然了，这一路上充满了不可思议的冒险，从故事的哪个部分开始呢？” 

possible actions:
1. "在意大利，我意外参与了一个传统的葡萄酒节..."
2. "在希腊的克里特岛，我们决定去附近的一个未标记的洞穴探险..."
3. "有一次，我在瑞士阿尔卑斯山中迷路了..."


...

I select option 3. TURN: 2 


在艺术品摊位前，你转向席琳，温柔地问：“你刚才提到了自己的梦想，能多告诉我一些吗？关于你对未来的想象。”席琳的眼神中闪过一丝惊喜，显然没想到你会这么认真地询问她的梦想。

她深吸了一口气，眼神变得坚定而充满梦想，“我...我真的很希望能成为一名设计师，或者...或者在艺术领域做一些大胆的尝试。”她的声音有一丝紧张和期待，“我一直对设计和创造性的工作充满热情。我父母希望我能成为一名老师，因为那更稳定，但我...我真的很想追求自己的梦想。”

听到席琳如此坦诚地分享她的梦想和内心的挣扎，你不由得对她充满了敬意。“我觉得你应该追求自己的梦想，”你鼓励她，“即使路上充满了挑战，但那才是属于你自己的路，不是吗？”

席琳听后，眼里闪烁着感激的光芒，她轻轻咬了咬嘴唇，然后露出了一个灿烂的笑容，“谢谢你，杰西。听你这么说，我感觉好多了。”她的笑容中带着释然和勇气，仿佛你的鼓励给了她前行的力量。

此时的气氛变得更加温暖，你们之间的关系也因此而更加紧密。分享梦想和彼此的理解，让这个晚上变得更加难忘。

possible actions:
1. 分享自己的梦想和对未来的看法。
2. 邀请席琳去附近的酒吧，听听音乐放松一下。
3. 继续探索广场，寻找更多有趣的艺术品。

I select option 2. TURN: 3 


在深入的交流后，你觉得是时候换个氛围了。你向席琳提议，“听说附近有个酒吧，今晚有音乐演出。我们去那儿放松一下如何？”席琳的眼睛里立刻闪现出兴奋的光芒，“哇，听起来真的很棒！我喜欢音乐，尤其是现场演出。”

你们一同前往那家酒吧。沿途上，席琳显得更加放松和自由，她和你分享她对音乐的喜爱，尤其是Etta James的音乐。“我觉得她的音乐有种魔力，能直击人心。”席琳说着，不自觉地随着记忆中的旋律轻轻摇摆。

到达酒吧时，氛围热烈而迷人。灯光昏暗却不失温馨，现场演出的音乐充满了魅力，让人不由自主地被吸引。你和席琳找了个靠窗的位置坐下，点了两杯酒。随着音乐的节奏，你们的对话也变得更加轻松愉快。

“真的很谢谢你今晚带我来这里，杰西。”席琳看着你，眼里满是感激，“这真是一个完美的夜晚。”她的笑容在昏暗的灯光下显得特别迷人，你不禁被她的魅力深深吸引。

“我也很高兴能和你一起享受这个夜晚，席琳。”你回应着，心中满是喜悦。此时此刻，你感觉和席琳之间的距离更近了一步，这个晚上，注定成为你们共同回忆中美好的一章。

随着音乐的落幕，你们也意犹未尽地离开了酒吧，相约未来还要一起探索更多的美妙夜晚。今晚的经历，让你们的关系更加深厚，也让你们对未来充满了期待。

game master:
-|context of the round|-
summary: 在这个回合中，杰西和席琳在维也纳的广场上一起探索艺术，并深入交流了彼此的梦想。通过共享个人的梦想和对未来的看法，他们的关系变得更加紧密。最后，他们一起去了附近的酒吧，享受了一个充满音乐和愉快对话的夜晚，让他们的相遇变得更加难忘。

酒吧内人声鼎沸，摇滚乐队的现场演出将氛围推至高潮。人群密集地挤在舞台前，挥舞着手臂，陶醉于音乐之中。

你们找了个角落的位置坐下，点了两杯酒。席琳注视着台上的摇滚乐队，身体不自觉地跟着律动摇摆起来。她的嘴角带笑，露出浅浅的酒窝，偶尔还轻声地跟着乐队的旋律哼唱。

“没想到你居然喜欢听摇滚。”你惊喜地发现她的另一面。“哈哈哈，不要被我的外表欺骗了。”她看向你，对你眨眨眼，“我的父母让我从小学古典钢琴，但我偷偷藏了Patti Smith全部的专辑。”

眼前这个女孩好像更鲜活了，也更可爱了。你看着眼前这个让你一见钟情的女孩，想问一些关键性的问题。

possible actions:
1. "你的初恋是在什么时候?"
2. "你...有男朋友吗?"
3. "有没有你最难忘的某一任?"


I select option 2. TURN: 1 


你犹豫了一下，但还是鼓起勇气，试图探查她的感情世界，"你...有男朋友吗？" 你尽力让自己的声音听起来轻松自然。

席琳微微一愣，然后低下头，轻轻摇了摇头。她咬了咬嘴唇，轻声回答你，"没有，我现在单身。" 声音中带着一丝不确定，似乎是在思考这个问题对她意味着什么。

她抬头看向你，眼神中带着一点探究，似乎也在试图了解你，"你呢？"

我也没有. TURN: 2 


你轻笑了一下，尽量保持轻松的语气，“我也没有。”你顿了一下，然后补充说，“其实，我刚从一段关系中走出来，来这次旅行是想给自己一点时间，你懂的，重新找回自己。”

席琳听后，脸上的表情变得柔和，她似乎对你的坦白感到同情，同时也有一种不易察觉的安心感。“我觉得你做得对。有时候，一个人的旅行是最好的疗愈。”她微笑着，眼神中流露出一丝敬佩。

她轻轻地把玻璃杯旋转在手中，然后说，“每个人的心里都有一段故事，不是吗？”